## INITIAL PREPARATION
Imports, root dir

#### Data investigation is not included here

In [1]:
DATA_DIR='playground-series-s3e4/'

In [15]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import datetime
import pickle

import expander
from training_lib import *

In [5]:
df=pd.read_csv(DATA_DIR+"/train.csv").iloc[:,1:] #drop ID
#dfOrig=pd.read_csv(DATA_DIR+"/orig.csv")

## DATASET PREPARATION
column drops, union, one-hoe encoding, normalization

In [9]:
#https://www.kaggle.com/competitions/playground-series-s3e2/discussion/377373 SMOTE

import category_encoders as ce #https://www.kaggle.com/competitions/playground-series-s3e2/discussion/377827

#toBins [('column1',bins1),('column2',bins2)] - turn each specified number fearure into <bin> separate values
#unite [['column1','column2','column3'],['column1','column5']] - unite columns, replace with mean target (y) order
#     should be good if rows are too few and feats are too many
#drop ['column1',2,3] - drop columns
#one-hot True - run get_dummies to turn any str feature into one-hot encoding
#norm - normalization, "minmax" - turn to 0..1

def col(df,column): #get column name for non-str column
    if type(column)!=str:
        return df.columns[column]
    return column

def prepareDataset(df, targetCol, datasetParams, dfTest=None, verb=0):
    #Prepare dataset, using customization parameters
    #also prepare test data (without y) if provided
    df_updated = df.copy() #copy original df not to spoil it
    test = not (dfTest is None)
    if test:
        dfTest_updated=dfTest.copy()
    params = datasetParams.copy()
    defaultParams = {"TO_BINS":[],"UNITE":[],"DROP":[],"KEEP":[],"ONE-HOT":False,"NORM":"None","ENCODING":[]}
    for p in defaultParams: #use default params for missing
        if not (p in params): params[p] = defaultParams[p]
    targetCol = col(df, targetCol)
    df_updated = df_updated.drop(targetCol, axis = 1) #drop target column, to add in the end
    if test and (targetCol in dfTest_updated.columns):
        dfTest_updated = dfTest_updated.drop(targetCol, axis = 1)
    for cols in [params["DROP"], params["KEEP"]]: #make num format for column names in parameters
        for i in range(len(cols)):
            if type(cols[i] == str):
                cols[i] = df.columns.get_loc(cols[i])
    #############################################
    for pair in params["TO_BINS"]: #slice these num feats by bins
        c = col(df_updated,pair[0])
        minVal, maxVal = df_updated[c].min(), df_updated[c].max()
        binSize = (maxVal-minVal) / pair[1]
        df_updated[c] = df_updated[c] - minVal
        df_updated[c] = df_updated[c].floordiv(binSize) * binSize
        if test:
            dfTest_updated[c] = dfTest_updated[c] - minVal
            dfTest_updated[c] = dfTest_updated[c].floordiv(binSize) * binSize
        if verb >= 2: #make histogram to check how consistent are values
            plt.figure()
            df_updated[c].hist()
    #############################################          
    for union in params["UNITE"]: #unite features
        cols = [] #get columns in one format
        newColumnName = "" #new column name from these columns
        for column in union:
            cols.append(df_updated.columns.get_loc(col(df_updated,column)))
            newColumnName += df_updated.columns[cols[-1]] + "_"
        newColumnName = newColumnName[:-1]
        newColumnInd = len(df_updated.columns)
        df_updated[newColumnName] = df_updated.iloc[:,0] #just add new column
        if test:
            dfTest_updated[newColumnName] = dfTest_updated.iloc[:,0] #just add new column

        uValues = {} #new unique values and mean target for them
        for row in range(df_updated.shape[0]):
            unionValue = ""
            for c in cols: #get new value from union
                unionValue += str(df_updated.iat[row,c])+"_"
            uValues[unionValue] = 0
            df_updated.iat[row,newColumnInd] = unionValue #put the new value to the new column
            
        if test: #same for submission or validation
            for row in range(dfTest_updated.shape[0]):
                unionValue=""
                for c in cols: #get new value from union
                    unionValue += str(dfTest_updated.iat[row,c])+"_"
                uValues[unionValue] = 0
                dfTest_updated.iat[row,newColumnInd] = unionValue #put the new value to the new column
                
        valList=[] #put mean targets in list to sort and get order (useless for decision trees)
        for val in uValues:
            newVal = df[df_updated[newColumnName] == val][targetCol].mean()
            uValues[val] = newVal
            valList.append((newVal,val))
        valList=sorted(valList)
        for p in valList:
            df_updated[newColumnName] = df_updated[newColumnName].replace(p[1],p[0])
            if test:
                dfTest_updated[newColumnName] = dfTest_updated[newColumnName].replace(p[1],p[0])
                
        if verb >= 2: #make histogram to check how consistent are values
            plt.figure()
            df_updated[newColumnName].hist()

        if verb>=1: #print count of new values, this shouldn't be too big for dataset
            print(newColumnName,len(uValues))
    #############################################        
    #perform columns drop
    df_updated = df_updated.drop(df_updated.columns[params["DROP"]], axis = 1)
    if test:
        dfTest_updated = dfTest_updated.drop(dfTest_updated.columns[params["DROP"]], axis = 1)
    #############################################        
    #perform columns keep
    if len(params["KEEP"]) > 0:
        df_updated = df_updated.iloc[:,params["KEEP"]]
        if test:
            dfTest_updated = dfTest_updated.iloc[:,params["KEEP"]]
    #############################################
    #perform encoding
    for encoding in params["ENCODING"]:
        if encoding[0] == "ORDINAL":
            print(encoding)
            #mapping=None
            #if len(encoding) > 2:
            #    mapping=encoding[2]
            encoder = ce.OrdinalEncoder(cols=[col(df_updated, c) for c in encoding[1]], return_df=True) #, mapping=mapping)
            df_updated = encoder.fit_transform(df_updated)
            if test:
                dfTest_updated = encoder.transform(dfTest_updated)
    #############################################    
    if params["ONE-HOT"]==True:
        df_updated = pd.get_dummies(df_updated)
        if test:
            dfTest_updated = pd.get_dummies(dfTest_updated)
    #############################################
    if params["NORM"]=="minmax":
        df_updated=(df_updated-df_updated.min())/(df_updated.max()-df_updated.min())
        if test:
            dfTest_updated=(dfTest_updated - df_updated.min()) / (df_updated.max() - df_updated.min())
        #sc = StandardScaler() #can be used later for something else
        #X[num_cols] = sc.fit_transform(X[num_cols])
    #############################################
    if verb>=3:
        display(df_updated.head(12))
    if not test:
        if verb >= 1:
            print(df_updated.columns)
        return (df_updated.values, df[targetCol].values)
    if test:
        if verb >= 1:
            print('train',df_updated.columns)
            print('val/sub',dfTest_updated.columns)
        return (df_updated.values, df[targetCol].values, dfTest_updated.values)
    
NEED_TEST = True #run test in case of RnD or param search
if NEED_TEST:
    datasetParams={"ENCODING":["ORDINAL",[1,3,"Gender"]]}
    X, Y = prepareDataset(df, df.columns[-1], datasetParams, verb=3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,0.742144,...,-0.292969,-0.334701,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98
1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,-0.109948,...,0.065717,0.054848,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00
2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,-0.042027,...,-0.021375,-0.326725,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69
3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,0.885838,...,-0.340089,-0.095514,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00
4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,0.692665,...,-0.203628,0.099157,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00
5,0.0,1.052469,0.099592,1.077244,1.370024,-0.471625,0.238658,-0.281837,0.055925,0.425970,...,0.002408,-0.013418,0.210670,-0.031069,0.131544,0.452867,-0.376528,0.086428,0.049083,35.97
6,0.0,1.206129,0.420150,-0.358910,1.037382,0.735224,-0.082449,0.568558,-0.158037,-0.479937,...,-0.009909,-0.366513,-1.113867,0.042265,-0.819477,0.473615,-0.639652,0.006812,0.016747,36.00
7,0.0,-1.074970,-0.297535,-0.884465,0.065341,-0.239956,0.349602,1.756880,0.054202,-0.860554,...,1.081409,0.624183,0.908389,-0.233799,-0.457045,0.551736,-0.064708,0.137524,0.203869,440.00
8,0.0,2.154065,-0.000479,-2.125939,-0.121096,0.918473,-0.208507,0.226055,-0.236828,0.314947,...,-0.156510,0.243716,0.780992,-0.148060,-0.280708,0.507044,-0.420133,-0.037652,-0.079346,1.00
9,0.0,2.172667,-0.704020,-1.447412,-0.715805,-0.530117,-1.007526,-0.436662,-0.176599,-0.426167,...,-0.185965,0.502916,1.442638,-0.063396,0.017817,0.270144,0.240989,-0.057164,-0.081458,10.00


Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')


## TRAINING FUNCTIONS
grid run, tensorflow, catboost, xgboost. Should be in training_lib, but RnD can be done here

### Grid run (upper as should be changed less frequently than training (or not))

In [35]:
#dirty

#getParamConfigList should be in training_lib
def runGridTest(df, datasetParams={}, trainingParams={}, libSpecificParams={}, kfolds=10, stopRightAway=False, dfAdditional=None):
    datasetParamsList = getParamConfigList(datasetParams)
    trainingParamsList = getParamConfigList(trainingParams)
    libSpecificParamsList = getParamConfigList(libSpecificParams)
    print(len(datasetParamsList),'dataset params configs')
    print(len(trainingParamsList),'training params configs')
    print(len(libSpecificParamsList),'lib specific params configs')
    print(len(datasetParamsList)*len(trainingParamsList)*len(libSpecificParamsList),'all params configs')
    print('datasetParams',datasetParamsList)
    print('trainingParams',trainingParamsList)
    print('libSpecificParams',libSpecificParamsList)
    results=[]
    for datasetParamConfig in datasetParamsList:
        
        if dfAdditional is None:
            X, Y = prepareDataset(df, df.columns[-1], datasetParamConfig) #prepare dataset with this param configurations
        else:
            X, Y, X2 = prepareDataset(df, df.columns[-1], datasetParamConfig, dfAdditional) #prepare dataset with this param configurations
            #_, Y2 = prepareDataset(dfAdditional, dfAdditional.columns[-1], datasetParamConfig) #TODO - how to use transform from initial? Or should I make common? Probably its better to use contest dataset and not to adjust to orig
            Y2 = dfAdditional.iloc[:,-1].values
            #df, targetCol, datasetParams, dfTest=None, verb=0
        for trainingParamConfig in trainingParamsList:
            for libSpecificParamConfig in libSpecificParamsList:
                totalRes=0
                AUCs=[]
                stf = StratifiedKFold(kfolds, shuffle=True, random_state=1) #make k-folds
                for train_ix, test_ix in tqdm(stf.split(X, Y), total = kfolds):
                    if False:
                        df2=df.iloc[train_ix]
                        subDf=df.iloc[test_ix]
                        X_Train, Y_Train, X_Test = prepareDataset(df2, df.columns[-1], datasetParamConfig, subDf)
                        
                        Y_Pred=trainingParamConfig["FUNC"](X_Train, Y_Train, X_Test, subDf.iloc[:,-1].values, X_Test, trainingParamConfig, libSpecificParamConfig) #run training and get predictions
                    else:
                        if dfAdditional is None:
                            Y_Pred=trainingParamConfig["FUNC"](X[train_ix], Y[train_ix], X[test_ix], Y[test_ix], X[test_ix], trainingParamConfig, libSpecificParamConfig) #run training and get predictions
                        else:
                            Y_Pred=trainingParamConfig["FUNC"](np.vstack([X[train_ix],X2]), np.hstack([Y[train_ix],Y2]), X[test_ix], Y[test_ix], X[test_ix], trainingParamConfig, libSpecificParamConfig) #run training and get predictions
                    #auc=roc_auc_score(Y[test_ix], Y_Pred) #calculate AUC
                    #print(auc,'pre-auc')
                    #Y_Pred=fixPredsForAUC(Y_Pred)
                    auc=roc_auc_score(Y[test_ix], Y_Pred) #calculate AUC
                    #print(auc,'auc')
                    totalRes+=auc
                    AUCs.append(auc)
                    predsFalse=[]
                    predsTrue=[]
                    for i in range(len(Y[test_ix])):
                        if Y[test_ix][i]==0:
                            predsFalse.append(Y_Pred[i])
                        else:
                            predsTrue.append(Y_Pred[i])
                    #plt.figure(figsize=(8,8))
                    #plt.hist(predsFalse,color='blue',bins=30)
                    #plt.hist(predsTrue*10,color='red',bins=30)
                    if stopRightAway:
                        break
                #plt.figure()
                #plt.hist(AUCs) #hist for AUCs in k-folds
                #plt.title(str(totalRes/kfolds)+" "+str(datasetParamConfig)+" "+str(trainingParamConfig))
                AUCs=np.array(AUCs)
                aucInfo="{:.4f}: ".format(totalRes/kfolds)+"{:.3f}".format(AUCs.min())+"-{:.3f}".format(AUCs.max())+" var: {:.5f}".format((AUCs-AUCs.mean()).var())
                print("kfolded auc",aucInfo,str(datasetParamConfig)+" "+str(trainingParamConfig)+" "+str(libSpecificParamConfig))
                results.append((totalRes/kfolds,str(datasetParamConfig)+" "+str(trainingParamConfig)+" "+str(libSpecificParamConfig)))
    return results

## Ensemble

In [6]:
#https://www.kaggle.com/competitions/playground-series-s3e2/discussion/377732

def ensembleTrain(X,Y,X_val,Y_val,X_test,trainingParams,libSpecificParams):
    params=trainingParams.copy()
    defaultParams={"ALG":"", "SPLIT":0.5}
    for p in defaultParams:
        if not (p in params): params[p] = defaultParams[p]
    
    totalWeight=0
    
    #simple blending OR train
    #"ALG", "VAL"
    #if alg - make one new validation split (better stratified)
    if (params["ALG"] != ""):
        n2 = int(len(X) * params["SPLIT"]) #examples to train ensemble
        X2 = X[-n2:]
        Y2 = Y[-n2:]
        X1 = X[:-n2]
        Y1 = Y[:-n2]
        totalPreds_train = np.zeros(len(X2))
        totalPreds_test = np.zeros(len(X_test))
    else:
        totalPreds=np.zeros(len(X_test))
    
    for method in trainingParams["METHODS"]:
        #print(method)
        #print(X.shape,X_val.shape,X_test.shape)
        weight = 1
        if len(method) >= 2: weight = method[2]
        totalWeight += weight
        if (params["ALG"] == ""):
            Y_Pred = method[0]["FUNC"](X, Y, X_val, Y_val, X_test, method[0], method[1])
            Y_Pred -= Y_Pred.min() #normalize #TODO RANKING
            Y_Pred /= Y_Pred.max() #normalize
            #print(totalPreds.shape,Y_Pred.shape)
            totalPreds += Y_Pred * weight    
        else:
            Y_Pred_train = method[0]["FUNC"](X1, Y1, X2, Y2, X2, method[0], method[1])
            Y_Pred_test = method[0]["FUNC"](X2, Y2, X_val, Y_val, X_test, method[0], method[1])
            Y_Pred_test -= Y_Pred_train.min()
            Y_Pred_train -= Y_Pred_train.min()
            Y_Pred_test /= Y_Pred_train.max()
            Y_Pred_train /= Y_Pred_train.max()
            totalPreds_train += Y_Pred_train * weight
            totalPreds_test += Y_Pred_test * weight
        
        
    if params["ALG"] != "": #perform meta-training for ensembling
        #Y_Pred1 = totalPreds / totalWeight
        #print(X.shape,X2.shape,Y_Pred1.shape)
        totalPreds_train /= totalWeight
        totalPreds_test /= totalWeight
        #print(totalPreds_train.shape,totalPreds_test.shape)
        Y_Pred = params["ALG"](totalPreds_train.reshape(-1,1), Y2, [], Y_val, totalPreds_test.reshape(-1,1), trainingParams, libSpecificParams)
        return Y_Pred
    else:    
        return totalPreds / totalWeight

### CatBoost, sklearn RandomForest. And other to do (xgboost, something else)...

In [23]:
#dirty

#catboost training functions
from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv

def catBoostTrain(X,Y,X_val,Y_val,X_test,trainingParams,libSpecificParams):
    params=trainingParams.copy()
    defaultParams={"ITERS":300,"LOSS":"Poisson","SEED":1} #"st_weight":1,
    for p in defaultParams:
        if not (p in params):
            params[p] = defaultParams[p]
    cat = CatBoostRegressor(iterations=params["ITERS"], loss_function=params["LOSS"], use_best_model=True, thread_count=2, random_seed=params["SEED"],**libSpecificParams) #, hints="skip_train~false")#, use_best_model=True) #class_weights={0:1,1:params["st_weight"]}, 
    cat_feats=[] #handle str and other features
    sample_weight=np.ones(len(Y))
    if ("CLASS_WEIGHT" in trainingParams) and (trainingParams["CLASS_WEIGHT"]>1):
        w=trainingParams["CLASS_WEIGHT"]
        for i in range(len(Y)):
            if Y[i]==1:
                sample_weight[i]=w
    for i in range(X.shape[1]):
        if type(X[0,i])==str:
            cat_feats.append(i)
    cat.fit(X, Y, cat_features=cat_feats, plot=False, verbose=False, eval_set=(X_val,Y_val), sample_weight=sample_weight) #train
    #return cat.predict_proba(X_test)[:,1] #return probabilities of 1
    return cat.predict(X_test)

#random forest training

from sklearn.ensemble import RandomForestClassifier

def randomForestTrain(X,Y,X_val,Y_val,X_test,trainingParams,libSpecificParams):
    rfc = RandomForestClassifier(random_state=1,**libSpecificParams) #n_estimators=3,max_depth=2,
    rfc.fit(X, Y.reshape(-1))
    return rfc.predict_proba(X_test)[:,1]

#xgboost

import xgboost as xgb

def XGBTrain(X,Y,X_val,Y_val,X_test,trainingParams,libSpecificParams): #trainX,indTrainX,trainY,valX,indValX,params):
    model=xgb.XGBRegressor(**libSpecificParams)
    model.fit(X,Y)
    preds=model.predict(X_test)
    return preds

#lightgbm

import lightgbm as lgbm

def LGBMTrain(X,Y,X_val,Y_val,X_test,trainingParams,libSpecificParams):
    model=lgbm.LGBMRegressor(**libSpecificParams)
    model.fit(X,Y)
    preds=model.predict(X_test)
    return preds
    
#Testing development
NEED_TEST = False
if NEED_TEST:
    datasetParams={"ONE-HOT":True}
    X, Y = prepareDataset(df, datasetParams) #prepare dataset with all the params
    rfc = RandomForestClassifier(random_state=1)
    rfc.fit(X[:-1000], Y[:-1000].reshape(-1))
    Y_Pred=rfc.predict_proba(X[-1000:])[:,1]
    auc=roc_auc_score(Y[-1000:], Y_Pred) #cal

### TensorFlow

In [51]:
#dirty and stupid lib version

#tensorflow training
from tensorflow.python.keras import layers as L
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import regularizers
import numpy as np

import tensorflow as tf
print('tensorflow version',tf.__version__)

def buildDense(columns,params):
    #Build dense neural network model
    inputLayer=L.Input(columns,name="Input")
    X=inputLayer
    for layer in params["LAYERS"]:
        X=L.Dense(layer,activation=None,activity_regularizer=params["REGULARIZATION"])(X)
        #X=L.BatchNormalization()(X)
        if params["DROPOUT"]>0:
            X=L.Dropout(params["DROPOUT"])(X)
        X=L.Activation(activation=params["ACTIVATION"])(X)
    output=L.Dense(1,activation="tanh")(X)
    model = Model(inputs=[inputLayer], outputs=[output])
    return model

def tensorFlowTrain(X,Y,X_val,Y_val,X_test,trainingParams,libSpecificParams):
    params=trainingParams.copy()
    defaultParams={"MODEL":buildDense,"LAYERS":[64],"ACTIVATION":"relu","DROPOUT":0,"REGULARIZATION":None,"LOSS":"mse","ITERS":50,"OPTIMIZER":"Adam","LR":0.001,"VERB":0}
    for p in defaultParams:
        if not (p in params):
            params[p] = defaultParams[p]
    model=params["MODEL"](X.shape[1],params)
    model.compile(loss=params["LOSS"], optimizer=params["OPTIMIZER"])
    K.set_value(model.optimizer.learning_rate, params["LR"])
    hist=model.fit(X,Y,epochs=params["ITERS"],validation_split=0.05,verbose=params["VERB"])
    preds=model.predict(X_test)
    return preds

NEED_TEST = False
if NEED_TEST:
    pass #code to run this

2023-01-20 19:59:53.660740: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-20 19:59:53.660763: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


tensorflow version 2.8.0


Using TensorFlow backend.


## RUNNING CONFIGS

#### Can be done in separate files for clarity

In [8]:
datasetParams={}
trainingParams={"FUNC":catBoostTrain}
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

<function catBoostTrain at 0x7f18d80b5ee0> <class 'function'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7f18d80b5ee0>}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [02:51<00:00, 17.19s/it]

kfolded auc 0.8205: 0.797-0.846 var: 0.00036 {} {'FUNC': <function catBoostTrain at 0x7f18d80b5ee0>} {}


In [13]:
datasetParams={"DROP":[["Time"]]}
trainingParams={"FUNC":catBoostTrain}
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

[['Time']] <class 'list'>
<function catBoostTrain at 0x7f18d80b5ee0> <class 'function'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'DROP': ['Time']}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7f18d80b5ee0>}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [02:48<00:00, 16.81s/it]

kfolded auc 0.8212: 0.802-0.853 var: 0.00028 {'DROP': [0]} {'FUNC': <function catBoostTrain at 0x7f18d80b5ee0>} {}


In [14]:
datasetParams={}
trainingParams={"FUNC":XGBTrain}
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

<function XGBTrain at 0x7f18d7dd6d30> <class 'function'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{}]
trainingParams [{'FUNC': <function XGBTrain at 0x7f18d7dd6d30>}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [07:55<00:00, 47.56s/it]

kfolded auc 0.7184: 0.654-0.780 var: 0.00175 {} {'FUNC': <function XGBTrain at 0x7f18d7dd6d30>} {}


In [15]:
datasetParams={}
trainingParams={"FUNC":LGBMTrain}
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

<function LGBMTrain at 0x7f18d74d1f70> <class 'function'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{}]
trainingParams [{'FUNC': <function LGBMTrain at 0x7f18d74d1f70>}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [00:33<00:00,  3.37s/it]

kfolded auc 0.7447: 0.681-0.841 var: 0.00229 {} {'FUNC': <function LGBMTrain at 0x7f18d74d1f70>} {}


In [27]:
df2=df.copy()
df2["Amount"]=np.log(df["Amount"].values+1)
datasetParams={"DROP":[["Time"]],"TO_BINS":[[("Amount",20)]]}
trainingParams={"FUNC":catBoostTrain}
libSpecificParams={}

res=runGridTest(df2,datasetParams,trainingParams,libSpecificParams,10)

[['Time']] <class 'list'>
[[('Amount', 20)]] <class 'list'>
<function catBoostTrain at 0x7f18d80b5ee0> <class 'function'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'DROP': ['Time'], 'TO_BINS': [('Amount', 20)]}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7f18d80b5ee0>}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [02:44<00:00, 16.48s/it]

kfolded auc 0.8210: 0.796-0.851 var: 0.00026 {'DROP': [0], 'TO_BINS': [('Amount', 20)]} {'FUNC': <function catBoostTrain at 0x7f18d80b5ee0>} {}


In [ ]:
df2=df.copy()
df2["Amount"]=np.log(df["Amount"].values)
datasetParams={"DROP":[["Time"]],"TO_BINS":[[("Amount",20)]]}
trainingParams={"FUNC":catBoostTrain}
libSpecificParams={}

res=runGridTest(df2,datasetParams,trainingParams,libSpecificParams,10)

In [37]:
df2=df.copy()

BINS=40

exp=expander.Normalizer1d(BINS)
am=np.log(df["Amount"].values+1)
exp.fit(am)
am2=exp.predict(am)
df2["Amount"]=am2

df2Add=dfOrig.copy()
am=np.log(df2Add["Amount"].values+1)
#exp.fit(am)
am2=exp.predict(am)
df2Add["Amount"]=am

datasetParams={"DROP":[["Time"]],"TO_BINS":[[("Amount",BINS)]]}
trainingParams={"FUNC":catBoostTrain}
libSpecificParams={}

res=runGridTest(df2,datasetParams,trainingParams,libSpecificParams,10,dfAdditional=df2Add)

[['Time']] <class 'list'>
[[('Amount', 40)]] <class 'list'>
<function catBoostTrain at 0x7fe8ef935f70> <class 'function'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'DROP': ['Time'], 'TO_BINS': [('Amount', 40)]}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7fe8ef935f70>}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [05:24<00:00, 32.47s/it]

kfolded auc 0.7989: 0.763-0.829 var: 0.00039 {'DROP': [0], 'TO_BINS': [('Amount', 40)]} {'FUNC': <function catBoostTrain at 0x7fe8ef935f70>} {}


In [ ]:
#^^^ additional is bad

In [46]:
df2=df.copy()

BINS=30

exp=expander.Normalizer1d(BINS)
am=np.log2(df["Amount"].values+1)
exp.fit(am)
am2=exp.predict(am)
df2["Amount"]=am2

datasetParams={"DROP":[["Time"]],"TO_BINS":[[("Amount",BINS)]]}
trainingParams={"FUNC":catBoostTrain}
libSpecificParams={}

res=runGridTest(df2,datasetParams,trainingParams,libSpecificParams,10)

[['Time']] <class 'list'>
[[('Amount', 30)]] <class 'list'>
<function catBoostTrain at 0x7fe8ef935f70> <class 'function'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'DROP': ['Time'], 'TO_BINS': [('Amount', 30)]}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7fe8ef935f70>}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [02:25<00:00, 14.56s/it]

kfolded auc 0.8195: 0.793-0.851 var: 0.00031 {'DROP': [0], 'TO_BINS': [('Amount', 30)]} {'FUNC': <function catBoostTrain at 0x7fe8ef935f70>} {}


In [ ]:
#kfolded auc 0.8221: 0.798-0.849 var: 0.00030 {'DROP': [0], 'TO_BINS': [('Amount', 100)]} {'FUNC': <function catBoostTrain at 0x7fe8ef935f70>} {}
#kfolded auc 0.8241: 0.798-0.855 var: 0.00028 {'DROP': [0], 'TO_BINS': [('Amount', 50)]} {'FUNC': <function catBoostTrain at 0x7fe8ef935f70>} {}
#kfolded auc 0.8224: 0.799-0.848 var: 0.00026 {'DROP': [0], 'TO_BINS': [('Amount', 40)]} {'FUNC': <function catBoostTrain at 0x7fe8ef935f70>} {} Stable
#kfolded auc 0.8195: 0.793-0.851 var: 0.00031 {'DROP': [0], 'TO_BINS': [('Amount', 30)]} {'FUNC': <function catBoostTrain at 0x7fe8ef935f70>} {}
#kfolded auc 0.8213: 0.798-0.850 var: 0.00029 {'DROP': [0], 'TO_BINS': [('Amount', 15)]} {'FUNC': <function catBoostTrain at 0x7fe8ef935f70>} {}
#kfolded auc 0.8214: 0.800-0.853 var: 0.00032 {'DROP': [0], 'TO_BINS': [('Amount', 10)]} {'FUNC': <function catBoostTrain at 0x7fe8ef935f70>} {}


In [38]:
datasetParams={"ONE-HOT":True}
trainingParams={"FUNC":[randomForestTrain,XGBTrain]}
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

True <class 'bool'>
[<function randomForestTrain at 0x7fca9b7f28b0>, <function XGBTrain at 0x7fca9937d700>] <class 'list'>
1 dataset params configs
2 training params configs
1 lib specific params configs
2 all params configs
datasetParams [{'ONE-HOT': True}]
trainingParams [{'FUNC': <function randomForestTrain at 0x7fca9b7f28b0>}, {'FUNC': <function XGBTrain at 0x7fca9937d700>}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [00:03<00:00,  3.15it/s]


kfolded auc 0.8128746093031808 {'ONE-HOT': True} {'FUNC': <function randomForestTrain at 0x7fca9b7f28b0>} {}


100%|███████████████████████████████████████████| 10/10 [00:10<00:00,  1.07s/it]

kfolded auc 0.742064487957345 {'ONE-HOT': True} {'FUNC': <function XGBTrain at 0x7fca9937d700>} {}


In [22]:
datasetParams={"ONE-HOT":True}
trainingParams={"FUNC":ensembleTrain,"METHODS":[[]]}
trainingParams["METHODS"][0].append(({"FUNC":randomForestTrain},{},1))
trainingParams["METHODS"][0].append(({"FUNC":catBoostTrain},{},3))
trainingParams["METHODS"][0].append(({"FUNC":XGBTrain},{},2))
#print()
#print(trainingParams["METHODS"][0])
#print()
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

True <class 'bool'>
<function ensembleTrain at 0x7f951908e4c0> <class 'function'>
[[({'FUNC': <function randomForestTrain at 0x7f964c257ee0>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7f95f7a22c10>}, {}, 3), ({'FUNC': <function XGBTrain at 0x7f95dc7c4280>}, {}, 2)]] <class 'list'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'ONE-HOT': True}]
trainingParams [{'FUNC': <function ensembleTrain at 0x7f951908e4c0>, 'METHODS': [({'FUNC': <function randomForestTrain at 0x7f964c257ee0>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7f95f7a22c10>}, {}, 3), ({'FUNC': <function XGBTrain at 0x7f95dc7c4280>}, {}, 2)]}]
libSpecificParams [{}]


  0%|                                                    | 0/10 [00:00<?, ?it/s]

({'FUNC': <function randomForestTrain at 0x7f964c257ee0>}, {}, 1)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function catBoostTrain at 0x7f95f7a22c10>}, {}, 3)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function XGBTrain at 0x7f95dc7c4280>}, {}, 2)
(1509, 54) (168, 54) (168, 54)


 10%|████▍                                       | 1/10 [00:10<01:30, 10.03s/it]

({'FUNC': <function randomForestTrain at 0x7f964c257ee0>}, {}, 1)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function catBoostTrain at 0x7f95f7a22c10>}, {}, 3)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function XGBTrain at 0x7f95dc7c4280>}, {}, 2)
(1509, 54) (168, 54) (168, 54)


...

({'FUNC': <function randomForestTrain at 0x7f964c257ee0>}, {}, 1)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function catBoostTrain at 0x7f95f7a22c10>}, {}, 3)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function XGBTrain at 0x7f95dc7c4280>}, {}, 2)
(1510, 54) (167, 54) (167, 54)


 90%|███████████████████████████████████████▌    | 9/10 [00:44<00:04,  4.02s/it]

({'FUNC': <function randomForestTrain at 0x7f964c257ee0>}, {}, 1)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function catBoostTrain at 0x7f95f7a22c10>}, {}, 3)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function XGBTrain at 0x7f95dc7c4280>}, {}, 2)
(1510, 54) (167, 54) (167, 54)


100%|███████████████████████████████████████████| 10/10 [00:48<00:00,  4.82s/it]

kfolded auc 0.8174000275785991 {'ONE-HOT': True} {'FUNC': <function ensembleTrain at 0x7f951908e4c0>, 'METHODS': [({'FUNC': <function randomForestTrain at 0x7f964c257ee0>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7f95f7a22c10>}, {}, 3), ({'FUNC': <function XGBTrain at 0x7f95dc7c4280>}, {}, 2)]} {}


In [81]:
datasetParams={"ONE-HOT":True}
trainingParams={"FUNC":ensembleTrain,"METHODS":[[]]} #,"ALG":tensorFlowTrain,"LAYERS":[[1]]}
trainingParams["METHODS"][0].append(({"FUNC":randomForestTrain},{},1))
trainingParams["METHODS"][0].append(({"FUNC":catBoostTrain},{},4))
trainingParams["METHODS"][0].append(({"FUNC":XGBTrain},{},2))
trainingParams["METHODS"][0].append(({"FUNC":LGBMTrain},{},2))
#print()
#print(trainingParams["METHODS"][0])
#print()
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

True <class 'bool'>
<function ensembleTrain at 0x7f8596fca280> <class 'function'>
[[({'FUNC': <function randomForestTrain at 0x7f8596fca310>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7f8596fca430>}, {}, 4), ({'FUNC': <function XGBTrain at 0x7f85d67c9ee0>}, {}, 2), ({'FUNC': <function LGBMTrain at 0x7f858b423dc0>}, {}, 2)]] <class 'list'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'ONE-HOT': True}]
trainingParams [{'FUNC': <function ensembleTrain at 0x7f8596fca280>, 'METHODS': [({'FUNC': <function randomForestTrain at 0x7f8596fca310>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7f8596fca430>}, {}, 4), ({'FUNC': <function XGBTrain at 0x7f85d67c9ee0>}, {}, 2), ({'FUNC': <function LGBMTrain at 0x7f858b423dc0>}, {}, 2)]}]
libSpecificParams [{}]


100%|███████████████████████████████████████████| 10/10 [00:22<00:00,  2.22s/it]

kfolded auc 0.8173: 0.756-0.905 var: 0.002 {'ONE-HOT': True} {'FUNC': <function ensembleTrain at 0x7f8596fca280>, 'METHODS': [({'FUNC': <function randomForestTrain at 0x7f8596fca310>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7f8596fca430>}, {}, 4), ({'FUNC': <function XGBTrain at 0x7f85d67c9ee0>}, {}, 2), ({'FUNC': <function LGBMTrain at 0x7f858b423dc0>}, {}, 2)]} {}


In [70]:
datasetParams={"ONE-HOT":True}
trainingParams={"FUNC":ensembleTrain,"SPLIT":0.3,"METHODS":[[]],"ALG":tensorFlowTrain,"LAYERS":[[8]],"LOSS":["Hinge","CategoricalHinge","CategoricalCrossentropy"]}
trainingParams["METHODS"][0].append(({"FUNC":randomForestTrain},{},1))
trainingParams["METHODS"][0].append(({"FUNC":catBoostTrain},{},4))
trainingParams["METHODS"][0].append(({"FUNC":XGBTrain},{},2))
trainingParams["METHODS"][0].append(({"FUNC":LGBMTrain},{},2))
#print()
#print(trainingParams["METHODS"][0])
#print()
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

True <class 'bool'>
<function ensembleTrain at 0x7fb9d1bb90d0> <class 'function'>
0.3 <class 'float'>
[[({'FUNC': <function randomForestTrain at 0x7fb9e01fdb80>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7fb9e01fd550>}, {}, 4), ({'FUNC': <function XGBTrain at 0x7fb9d1ba6d30>}, {}, 2), ({'FUNC': <function LGBMTrain at 0x7fb9d1ba6790>}, {}, 2)]] <class 'list'>
<function tensorFlowTrain at 0x7fba2f801280> <class 'function'>
[[8]] <class 'list'>
['Hinge', 'CategoricalHinge', 'CategoricalCrossentropy'] <class 'list'>
1 dataset params configs
3 training params configs
1 lib specific params configs
3 all params configs
datasetParams [{'ONE-HOT': True}]
trainingParams [{'FUNC': <function ensembleTrain at 0x7fb9d1bb90d0>, 'SPLIT': 0.3, 'METHODS': [({'FUNC': <function randomForestTrain at 0x7fb9e01fdb80>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7fb9e01fd550>}, {}, 4), ({'FUNC': <function XGBTrain at 0x7fb9d1ba6d30>}, {}, 2), ({'FUNC': <function LGBMTrain at 0x7fb9d1ba6790>}, {}, 2)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

({'FUNC': <function randomForestTrain at 0x7fb9e01fdb80>}, {}, 1)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function catBoostTrain at 0x7fb9e01fd550>}, {}, 4)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function XGBTrain at 0x7fb9d1ba6d30>}, {}, 2)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function LGBMTrain at 0x7fb9d1ba6790>}, {}, 2)
(1509, 54) (168, 54) (168, 54)
(452,) (168,)


 10%|████▍                                       | 1/10 [00:06<00:57,  6.44s/it]

({'FUNC': <function randomForestTrain at 0x7fb9e01fdb80>}, {}, 1)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function catBoostTrain at 0x7fb9e01fd550>}, {}, 4)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function XGBTrain at 0x7fb9d1ba6d30>}, {}, 2)
(1509, 54) (168, 54) (168, 54)
({'FUNC': <function LGBMTrain at 0x7fb9d1ba6790>}, {}, 2)
(1509, 54) (168, 54) (168, 54)
(452,) (168,)


...

({'FUNC': <function randomForestTrain at 0x7fb9e01fdb80>}, {}, 1)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function catBoostTrain at 0x7fb9e01fd550>}, {}, 4)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function XGBTrain at 0x7fb9d1ba6d30>}, {}, 2)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function LGBMTrain at 0x7fb9d1ba6790>}, {}, 2)
(1510, 54) (167, 54) (167, 54)
(453,) (167,)


 90%|███████████████████████████████████████▌    | 9/10 [01:27<00:08,  8.03s/it]

({'FUNC': <function randomForestTrain at 0x7fb9e01fdb80>}, {}, 1)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function catBoostTrain at 0x7fb9e01fd550>}, {}, 4)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function XGBTrain at 0x7fb9d1ba6d30>}, {}, 2)
(1510, 54) (167, 54) (167, 54)
({'FUNC': <function LGBMTrain at 0x7fb9d1ba6790>}, {}, 2)
(1510, 54) (167, 54) (167, 54)
(453,) (167,)


100%|███████████████████████████████████████████| 10/10 [01:34<00:00,  9.45s/it]

kfolded auc 0.37268730465159033 {'ONE-HOT': True} {'FUNC': <function ensembleTrain at 0x7fb9d1bb90d0>, 'SPLIT': 0.3, 'METHODS': [({'FUNC': <function randomForestTrain at 0x7fb9e01fdb80>}, {}, 1), ({'FUNC': <function catBoostTrain at 0x7fb9e01fd550>}, {}, 4), ({'FUNC': <function XGBTrain at 0x7fb9d1ba6d30>}, {}, 2), ({'FUNC': <function LGBMTrain at 0x7fb9d1ba6790>}, {}, 2)], 'ALG': <function tensorFlowTrain at 0x7fba2f801280>, 'LAYERS': [8], 'LOSS': 'CategoricalCrossentropy'} {}


In [ ]:
datasetParams={"KEEP":[["MartialStatus","OverTime","StockOptionLevel","YearsInCurrentRole","YearsAtCompany","EnvironmentSatisfaction","JobLevel","JobRole","MonthlyIncome","YearsWithCurrManager","YearsSinceLastPromotion","TotalWorkingYears","JobSatisfaction"]],"UNITE":[]}
trainingParams={"FUNC":catBoostTrain,"LOSS":["Poisson","LogLinQuantile","RMSE"],"ITERS":[500,1000]}
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)

In [14]:
#checking all possible column drops
import pickle

drops=[[]]
for a1 in range(len(df.columns)-1):
    drops.append([a1])
    for a2 in range(a1+1,len(df.columns)-1):
        drops.append([a1,a2])
        '''for a3 in range(a2+1,len(df.columns)-1):
            drops.append([a1,a2,a3])
            for a4 in range(a3+1,len(df.columns)-1):
                drops.append([a1,a2,a3,a4])
                for a5 in range(a4+1,len(df.columns)-1):
                    drops.append([a1,a2,a3,a4,a5])
                    for a6 in range(a5+1,len(df.columns)-1):
                        drops.append([a1,a2,a3,a4,a5,a6])
                        for a7 in range(a6+1,len(df.columns)-1):
                            drops.append([a1,a2,a3,a4,a5,a6,a7])
                            for a8 in range(a7+1,len(df.columns)-1):
                                drops.append([a1,a2,a3,a4,a5,a6,a7,a8])
                                for a9 in range(a8+1,len(df.columns)-1):
                                    drops.append([a1,a2,a3,a4,a5,a6,a7,a8,a9])''';
print(len(drops))
datasetParams={"DROP":drops}
trainingParams={"FUNC":catBoostTrain,"ITERS":10}
libSpecificParams={}

res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)
with open("res.pickle","wb") as f:
    pickle.dump(res,f)

562
[[], [0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [0, 19], [0, 20], [0, 21], [0, 22], [0, 23], [0, 24], [0, 25], [0, 26], [0, 27], [0, 28], [0, 29], [0, 30], [0, 31], [0, 32], [1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [1, 15], [1, 16], [1, 17], [1, 18], [1, 19], [1, 20], [1, 21], [1, 22], [1, 23], [1, 24], [1, 25], [1, 26], [1, 27], [1, 28], [1, 29], [1, 30], [1, 31], [1, 32], [2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [2, 11], [2, 12], [2, 13], [2, 14], [2, 15], [2, 16], [2, 17], [2, 18], [2, 19], [2, 20], [2, 21], [2, 22], [2, 23], [2, 24], [2, 25], [2, 26], [2, 27], [2, 28], [2, 29], [2, 30], [2, 31], [2, 32], [3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [3, 10], [3, 11], [3, 12], [3, 13], [3, 14], [3, 15], [3, 16], [3, 17], [3, 18], [3, 19], [3, 20], [3, 21], [3

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 11.18it/s]


kfolded auc 0.7500981338481337 {'DROP': []} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 13.14it/s]


kfolded auc 0.7535911932340503 {'DROP': [0]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}


...

kfolded auc 0.7219201599558742 {'DROP': [31]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 10.06it/s]


kfolded auc 0.7636075105717962 {'DROP': [31, 32]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 10.07it/s]

kfolded auc 0.7186982901268616 {'DROP': [32]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}


In [15]:
res=sorted(res)
print(res[-10:])

[(0.7887219617576761, "{'DROP': [2, 12]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}"), (0.7917105626034198, "{'DROP': [19, 22]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}"), (0.7917105626034198, "{'DROP': [22, 24]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}"), (0.7917105626034198, "{'DROP': [22]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}"), (0.7917105626034198, "{'DROP': [7, 22]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}"), (0.7917211343997058, "{'DROP': [0, 14]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}"), (0.7949342710056995, "{'DROP': [7, 9]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}"), (0.7949342710056995, "{'DROP': [9, 19]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10} {}"), (0.7949342710056995, "{'DROP': [9, 24]} {'FUNC': <function catBoostTrain at 0x7fb474529dc0>, 'ITERS': 10}

## FINE-TUNING BEST CONFIGS (with Optuna)

In [84]:
import optuna

def my_objective(trial):
    param = {
 #     'max_leaves': trial.suggest_int('max_leaves', 20, 100),
      'max_depth': trial.suggest_int('max_depth', 2, 10, 2),
      'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 19, 3)
    }
    #datasetParams={"DROP":[[[1,3,5,6,8,9,11,12,13,14,15,18,20,22,25,27,28, 7,19,24]]],"UNITE":[[[1,3,8],[5,6],[12,9,20],[11,28],[13,15],[22,27],[18,14]]]} #drop 1-variant columns, unite (and drop) few-variants columns
    #trainingParams={"FUNC":catBoostTrain,"LOSS":"Poisson","ITERS":2000}
    datasetParams={}
    trainingParams={"FUNC":catBoostTrain,"LOSS":Poisson","ITERS":2000}
    libSpecificParams=param
    res=runGridTest(df,datasetParams,trainingParams,libSpecificParams,10)
    #res=np.array(res)
    return res[0][0]
    

cat_roc_study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42), #TPESampler(seed=42),
                                   pruner=optuna.pruners.MedianPruner( #HyperbandPruner(
      #  n_startup_trials=5, n_warmup_steps=30, interval_steps=6
    ))
cat_roc_study.optimize(my_objective, n_trials=12)

cat_roc_best = cat_roc_study.best_trial
cat_roc_best_params = cat_roc_best.params
print('score: {0}, params: {1}'.format(cat_roc_best.value, cat_roc_best_params))

[I 2023-01-17 21:38:46,930] A new study created in memory with name: no-name-d43ab61f-211c-44d8-afbb-e7c81d5dddde


[[[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]]] <class 'list'>
[[[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]] <class 'list'>
<function catBoostTrain at 0x7fca9b7f2ca0> <class 'function'>
Poisson <class 'str'>
2000 <class 'int'>
4 <class 'int'>
19 <class 'int'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'DROP': [[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]], 'UNITE': [[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7fca9b7f2ca0>, 'LOSS': 'Poisson', 'ITERS': 2000}]
libSpecificParams [{'max_depth': 4, 'min_data_in_leaf': 19}]


/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
  0%|                                                    | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
 10%|████▍                                       | 1/10 [00:04<00:39,  4.37s/it]/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tu

kfolded auc 0.8074616197830483 {'DROP': [[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]], 'UNITE': [[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]} {'FUNC': <function catBoostTrain at 0x7fca9b7f2ca0>, 'LOSS': 'Poisson', 'ITERS': 2000} {'max_depth': 4, 'min_data_in_leaf': 19}
[[[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]]] <class 'list'>
[[[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]] <class 'list'>
<function catBoostTrain at 0x7fca9b7f2ca0> <class 'function'>
Poisson <class 'str'>
2000 <class 'int'>
8 <class 'int'>
13 <class 'int'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'DROP': [[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]], 'UNITE': [[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7fca9b7f2ca0

/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
  0%|                                                    | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
 10%|████▍                                       | 1/10 [00:12<01:54, 12.75s/it]/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tu

...

kfolded auc 0.8093594870380585 {'DROP': [[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]], 'UNITE': [[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]} {'FUNC': <function catBoostTrain at 0x7fca9b7f2ca0>, 'LOSS': 'Poisson', 'ITERS': 2000} {'max_depth': 6, 'min_data_in_leaf': 7}
[[[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]]] <class 'list'>
[[[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]] <class 'list'>
<function catBoostTrain at 0x7fca9b7f2ca0> <class 'function'>
Poisson <class 'str'>
2000 <class 'int'>
8 <class 'int'>
1 <class 'int'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'DROP': [[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]], 'UNITE': [[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7fca9b7f2ca0>,

/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
  0%|                                                    | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
 10%|████▍                                       | 1/10 [00:12<01:56, 12.98s/it]/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tu

kfolded auc 0.8103504780290495 {'DROP': [[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]], 'UNITE': [[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]} {'FUNC': <function catBoostTrain at 0x7fca9b7f2ca0>, 'LOSS': 'Poisson', 'ITERS': 2000} {'max_depth': 8, 'min_data_in_leaf': 1}
[[[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]]] <class 'list'>
[[[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]] <class 'list'>
<function catBoostTrain at 0x7fca9b7f2ca0> <class 'function'>
Poisson <class 'str'>
2000 <class 'int'>
4 <class 'int'>
7 <class 'int'>
1 dataset params configs
1 training params configs
1 lib specific params configs
1 all params configs
datasetParams [{'DROP': [[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]], 'UNITE': [[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]}]
trainingParams [{'FUNC': <function catBoostTrain at 0x7fca9b7f2ca0>,

/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
  0%|                                                    | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
 10%|████▍                                       | 1/10 [00:05<00:53,  5.91s/it]/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:5382: FutureWarning: Using a non-tu

kfolded auc 0.8074616197830483 {'DROP': [[1, 3, 5, 6, 8, 9, 11, 12, 13, 14, 15, 18, 20, 22, 25, 27, 28, 7, 19, 24]], 'UNITE': [[1, 3, 8], [5, 6], [12, 9, 20], [11, 28], [13, 15], [22, 27], [18, 14]]} {'FUNC': <function catBoostTrain at 0x7fca9b7f2ca0>, 'LOSS': 'Poisson', 'ITERS': 2000} {'max_depth': 4, 'min_data_in_leaf': 7}
score: 0.8111907060121346, params: {'max_depth': 10, 'min_data_in_leaf': 4}


## SUBMISSION CREATION

In [47]:
#Submission file craft

dfTest=pd.read_csv(DATA_DIR+"/test.csv")
ids=dfTest.iloc[:,0]
dfTest=dfTest.iloc[:,1:] #drop ID

#Best configs
df2=df.copy()

BINS=40

exp = expander.Normalizer1d(BINS)
am = np.log2(df["Amount"].values+1)
exp.fit(am)
am2 = exp.predict(am)
df2["Amount"] = am2

am=np.log2(dfTest["Amount"].values+1)
am2=exp.predict(am)
dfTest["Amount"]=am2

datasetParams={"DROP":["Time"],"TO_BINS":[("Amount",BINS)]}
trainingParams={"FUNC":catBoostTrain}
libSpecificParams={}


X_Train, Y_Train, X_Test = prepareDataset(df2, df2.columns[-1], datasetParams, dfTest)

Y_Pred_total = 0
kCount = 0

stf = StratifiedKFold(12, shuffle=True, random_state=1)
for train_ix, test_ix in tqdm(stf.split(X_Train, Y_Train), total = 12): #get one stratified k-fold to estimate performance
    Y_Pred = catBoostTrain(X_Train[train_ix], Y_Train[train_ix], X_Train[test_ix], Y_Train[test_ix], X_Test, trainingParams, libSpecificParams)
    Y_Pred -= Y_Pred.min()
    Y_Pred /= Y_Pred.max()
    Y_Pred_total += Y_Pred
    kCount += 1
    #break

Y_Pred = Y_Pred_total / kCount #OOF
    
dfSub=pd.DataFrame({df.columns[-1]:Y_Pred},index=ids)
dfSub.to_csv(DATA_DIR+'/subs/sub_'+datetime.datetime.now().strftime("%d.%m.%Y__%H.%M.%S")+'.csv')
dfSub.head(12)

100%|███████████████████████████████████████████| 12/12 [02:54<00:00, 14.55s/it]


,Class
id,
219129,0.001773
219130,0.001235
219131,0.000833
219132,0.004191
219133,0.001885
219134,0.001510
219135,0.001502
219136,0.003089
219137,0.001309


In [15]:
df.columns

Index(['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeCount',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Attrition'],
      dtype='object')

In [20]:
#RnD
#print(df["JobInvolvement"].unique())
#df["RelationshipSatisfaction"].unique()
for c in df.columns:
    if len(df[c].unique())<7 and len(df[c].unique())>1:
        print(c,df[c].unique())

BusinessTravel ['Travel_Frequently' 'Travel_Rarely' 'Non-Travel']
Department ['Research & Development' 'Sales' 'Human Resources']
Education [ 3  4  1  2  5 15]
EducationField ['Medical' 'Other' 'Marketing' 'Life Sciences' 'Technical Degree'
 'Human Resources']
EnvironmentSatisfaction [4 1 3 2]
Gender ['Male' 'Female']
JobInvolvement [3 1 2 4]
JobLevel [1 2 5 3 4 7]
JobSatisfaction [4 1 2 3]
MaritalStatus ['Married' 'Divorced' 'Single']
OverTime ['Yes' 'No']
PerformanceRating [3 4]
RelationshipSatisfaction [2 4 3 1]
StockOptionLevel [1 2 0 3]
WorkLifeBalance [3 1 2 4]
Attrition [0 1]


In [48]:
#base Percents
print(df["Attrition"][df["Attrition"]==1].count()/df.shape[0])

0.11926058437686345


In [88]:
c1="JobLevel"
l1=df[c1].unique()
#c2="WorkLifeBalance"
#l2=df[c2].unique()
for i in range(len(l1)):
    total=df[c1][df[c1]==l1[i]].count()
    attr=df[c1][(df[c1]==l1[i]) & (df["Attrition"]==1)].count()
    result=""
    if total<40:
        result+=">"+str(l1[i])+"< SMALL!!!"
    print(total,"{:.2f}%".format(100*attr/total),result)
    

655 19.39% 
617 6.81% 
75 6.67% 
235 8.94% 
94 5.32% 
1 0.00% >7< SMALL!!!


In [101]:
feats=[]
for c in df.columns:
    if len(df[c].unique())<7 and len(df[c].unique())>1:
        print(c,df[c].unique())
        feats.append(c)

BusinessTravel ['Travel_Frequently' 'Travel_Rarely' 'Non-Travel']
Department ['Research & Development' 'Sales' 'Human Resources']
Education [ 3  4  1  2  5 15]
EducationField ['Medical' 'Other' 'Marketing' 'Life Sciences' 'Technical Degree'
 'Human Resources']
EnvironmentSatisfaction [4 1 3 2]
Gender ['Male' 'Female']
JobInvolvement [3 1 2 4]
JobLevel [1 2 5 3 4 7]
JobSatisfaction [4 1 2 3]
MaritalStatus ['Married' 'Divorced' 'Single']
OverTime ['Yes' 'No']
PerformanceRating [3 4]
RelationshipSatisfaction [2 4 3 1]
StockOptionLevel [1 2 0 3]
WorkLifeBalance [3 1 2 4]
Attrition [0 1]


In [121]:
rats={}
c1="JobLevel"
l1=df[c1].unique()
for c2 in feats:
    if c2==c1:
        continue
    #c2="EducationField"
    l2=df[c2].unique()
    for i in range(len(l1)):
        if not l1[i] in rats:
            rats[l1[i]]=[]
        ratsNow=[]
        for j in range(len(l2)):
            total=df[(df[c1]==l1[i]) & (df[c2]==l2[j])].shape[0]
            attr=df[(df[c1]==l1[i]) & (df[c2]==l2[j]) & (df["Attrition"]==1)].shape[0]
            result=""
            if total<20:
                result+="SMALL!!!"
            ratio=100*attr/max(1,total)
            print(total,attr,"{:.2f}%".format(ratio), l1[i],l2[j],result)
            if total>20:
                ratsNow.append(ratio)
        rats[l1[i]].extend(ratsNow)
print()
res=[]
for r in rats:
    ar=np.array(rats[r])
    if len(ar)==0:
        res.append((0,r))
    else:
        res.append(((ar-ar.mean()).std(),r))
res=sorted(res)
print(c1,c2,'std',res)

106 35 33.02% 1 Travel_Frequently 
512 89 17.38% 1 Travel_Rarely 
37 3 8.11% 1 Non-Travel 
92 8 8.70% 2 Travel_Frequently 
462 32 6.93% 2 Travel_Rarely 
63 2 3.17% 2 Non-Travel 
12 2 16.67% 5 Travel_Frequently SMALL!!!
58 3 5.17% 5 Travel_Rarely 
5 0 0.00% 5 Non-Travel SMALL!!!
39 4 10.26% 3 Travel_Frequently 
178 17 9.55% 3 Travel_Rarely 
18 0 0.00% 3 Non-Travel SMALL!!!
12 2 16.67% 4 Travel_Frequently SMALL!!!
79 3 3.80% 4 Travel_Rarely 
3 0 0.00% 4 Non-Travel SMALL!!!
0 0 0.00% 7 Travel_Frequently SMALL!!!
1 0 0.00% 7 Travel_Rarely SMALL!!!
0 0 0.00% 7 Non-Travel SMALL!!!
555 94 16.94% 1 Research & Development 
80 26 32.50% 1 Sales 
20 7 35.00% 1 Human Resources 
350 16 4.57% 2 Research & Development 
258 26 10.08% 2 Sales 
9 0 0.00% 2 Human Resources SMALL!!!
54 2 3.70% 5 Research & Development 
19 3 15.79% 5 Sales SMALL!!!
2 0 0.00% 5 Human Resources SMALL!!!
144 10 6.94% 3 Research & Development 
86 11 12.79% 3 Sales 
5 0 0.00% 3 Human Resources SMALL!!!
63 4 6.35% 4 Research & D

In [123]:
print(rats[4])

[3.7974683544303796, 6.349206349206349, 3.5714285714285716, 3.0303030303030303, 0.0, 9.67741935483871, 4.081632653061225, 3.125, 0.0, 1.9607843137254901, 9.30232558139535, 3.225806451612903, 9.523809523809524, 2.9411764705882355, 10.714285714285714, 6.666666666666667, 6.666666666666667, 9.523809523809524, 4.109589041095891, 4.938271604938271, 2.6315789473684212, 10.714285714285714, 2.272727272727273, 10.256410256410257, 5.660377358490566, 4.545454545454546, 0.0]
